In [1]:
from __future__ import print_function
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles

## Three visits in 2 filters with specified filter pair and time gap


for Fast transients identification

In [2]:
from lsst.sims.maf.metrics import BaseMetric


In [3]:
class threeVisitsWColorMetric(BaseMetric):
        

    def __init__(self, filterCol='filter', nightCol='night', 
                 mjdCol='observationStartMJD', dt1=0.5/24, dt2=1.5/24,
                 filters='gi', nVisitsMin=3, **kwargs):
        '''
        Calculates fields for which there are 3 visits in 2 filters with color within dt1 
        and shape at least separated by dt2
        
        returns 1 if the conditionas are all verified for the data slice, -666 otherwise
        default: dt1 30 min dt2 1.5 hours
        default filters y+g (cause there are a few)'''
        
        # Pass the same bins to the plotter.                                                      
        self.mjdCol = mjdCol
        self.nightCol = nightCol
        self.filterCol = filterCol        
        self.nVisitsMin = nVisitsMin
        self.dt1 = dt1
        self.dt2 = dt2
        self.filters = (filters[0], filters[1])
        super(threeVisitsWColorMetric, 
              self).__init__(col=[self.mjdCol, 
                                self.nightCol, 
                                self.filterCol], 
                                metricDtype='int', 
                                units='Fraction', 
                                **kwargs)

    def run(self, dataSlice, slicePoint=None):
        mins = [99, 99]
        maxs = [-99, -99]
        sorting = np.argsort(dataSlice[self.mjdCol])
        dataSlice = dataSlice[sorting]
        #count the observations within a night for each slice
        n, counts = np.unique(dataSlice[self.nightCol], return_counts=True)
        #print(dataSlice)
        minobs = n[counts >= 3]
        #print(n, counts, minobs)
        dt1, dt2, filtshape = 100, -1, 'x'

        metricval = 0

        #iterate over nights with more than 3 observations
        for i,night in enumerate(minobs):
            #this is a night with >=3 observations
            good = dataSlice[self.nightCol] == night
            
            #check if it has 2 filters at least
            n, count = np.unique(dataSlice[self.filterCol][good], return_counts=True)
            if len(n) > 1:
                #print(dataSlice[self.filterCol][good])
                #print(n, count)
                #obs with filter 1
                f0 = dataSlice[self.filterCol][good] == self.filters[0]
                #obs with filter 2
                f1 = dataSlice[self.filterCol][good] == self.filters[1]
                #print(self.filters[0], f0.sum(), self.filters[1], f1.sum())
                
                if f0.sum() == 0 or f1.sum() == 0:
                    metricval = 0.
                    return metricval
                
                if f0.sum() > 1:
                    #max gap with same filter
                    dt2 = dataSlice[self.mjdCol][good][f0][-1] - \
                               dataSlice[self.mjdCol][good][f0][0]
                    #print("dt2", dt2)
                    filtshape = self.filters[0]
                if f1.sum() > 1:
                    #max gap with same filter
                    _tmp = dataSlice[self.mjdCol][good][f1][-1] - \
                               dataSlice[self.mjdCol][good][f1][0]
                    #print("other dt2", _tmp)
                    if  _tmp > dt2:
                            filtshape = self.filters[1]
                            dt2 = _tmp
                dt1 = np.min([np.abs(t1-t2) for t1 in dataSlice[self.mjdCol][good][f0] 
                             for t2 in dataSlice[self.mjdCol][good][f1]])

                #print (dt1 , "(", self.dt1, ")", dt2, "(", self.dt2, ")", filtshape)
               
            else:
                # no 3 obs night
                return metricval
        #see if obs verify  gap min and max conditions
        if dt1 <= self.dt1 and dt2 >= self.dt2:
            metricval = 1.0
        return metricval
    
    def reducep(self, metricval):
        result = np.sum(metricval)
        return result


In [4]:
metric = threeVisitsWColorMetric(filters='ri')
slicer = slicers.HealpixSlicer(nside=8)
sqlconstraint = 'night < 365'
myBundle = metricBundles.MetricBundle(metric, slicer, sqlconstraint)

opsdb = db.OpsimDatabase('pontus_2564.db')
myBundle.setPlotDict({'colorMin': 0.0, 'colorMax': 1.8})
bgroup = metricBundles.MetricBundleGroup(
    {0: myBundle}, opsdb, outDir='newmetric_test', resultsDb=None)
bgroup.runAll()
bgroup.plotAll(closefigs=False)


Healpix slicer using NSIDE=8, approximate resolution 439.742261 arcminutes


OSError: Sqlite database file "pontus_2564.db" not found.

In [5]:
!wget  http://astro-lsst-01.astro.washington.edu:8081/db_gzip/pontus_2564.db.gz
        

--2018-11-28 12:38:38--  http://astro-lsst-01.astro.washington.edu:8081/db_gzip/pontus_2564.db.gz
Resolving astro-lsst-01.astro.washington.edu (astro-lsst-01.astro.washington.edu)... 140.142.4.21
Connecting to astro-lsst-01.astro.washington.edu (astro-lsst-01.astro.washington.edu)|140.142.4.21|:8081... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2043591549 (1.9G) [application/gzip]
Saving to: 'pontus_2564.db.gz.1'

100%[====================================>] 2,043,591,549 2.23MB/s   in 24m 59s

2018-11-28 13:03:41 (1.30 MB/s) - 'pontus_2564.db.gz.1' saved [2043591549/2043591549]



In [2]:
!gunzip pontus_2564.db.gz

^C
